
# Previsão da Estrutura Secundária de Proteínas Usando Redes Neurais

Este projeto visa prever a **estrutura secundária de proteínas** a partir de suas sequências de aminoácidos utilizando **redes neurais**. A previsão da estrutura secundária de proteínas (PSSP) é uma etapa importante no entendimento da forma tridimensional das proteínas, o que está intimamente relacionado à sua função biológica. A estrutura secundária de proteínas consiste principalmente em três elementos: **hélices alfa (α-helix)**, **folhas beta (β-sheet)** e **coils** (enrolamentos).

O projeto se concentra em utilizar técnicas de **deep learning** para criar um modelo capaz de classificar aminoácidos em uma dessas três classes estruturais com base na sequência fornecida. Para isso, o projeto utiliza redes neurais recorrentes, especificamente a arquitetura **LSTM (Long Short-Term Memory)**, que é conhecida por lidar bem com sequências de dados, como as sequências de aminoácidos nas proteínas.

# Principais Etapas do Projeto:

1. **Importação das Bibliotecas**:
   O projeto utiliza bibliotecas como **Pandas**, **NumPy**, e ferramentas de **Machine Learning** e **Deep Learning** da **Scikit-learn** e **TensorFlow**. Essas bibliotecas são essenciais para o processamento de dados, criação do modelo, e treinamento.

2. **Pré-processamento dos Dados**:
   - **Codificação One-Hot**: As sequências de aminoácidos são convertidas em uma representação de vetor conhecida como **codificação one-hot**, onde cada aminoácido é representado por um vetor binário, facilitando a compreensão do modelo.
   - **Padding de Sequências**: Como as proteínas têm tamanhos variados, as sequências são padronizadas para um comprimento fixo, adicionando um caractere de preenchimento (geralmente 'X') quando necessário. Isso permite que todas as entradas tenham o mesmo tamanho, o que é necessário para o treinamento da rede neural.

3. **Definição da Arquitetura da Rede Neural**:
   O modelo usa uma rede neural do tipo **LSTM**. Esta é uma variação das redes neurais recorrentes, capaz de aprender a partir de dependências de longo prazo em dados sequenciais, como ocorre em sequências de aminoácidos.
   - **Camadas do Modelo**: A rede contém camadas LSTM e camadas densas totalmente conectadas. A camada final usa a função de ativação **softmax** para classificar os aminoácidos nas três possíveis estruturas secundárias.

4. **Treinamento do Modelo**:
   - Os dados são divididos em **conjunto de treino** e **conjunto de teste**, e o modelo é treinado para minimizar o erro na previsão das estruturas secundárias.
   - Durante o treinamento, o modelo ajusta seus parâmetros para melhorar sua capacidade de prever corretamente a estrutura secundária de uma sequência desconhecida de aminoácidos.

5. **Avaliação do Modelo**:
   Após o treinamento, o desempenho do modelo é avaliado com base em métricas como **acurácia**, que indica a porcentagem de previsões corretas, entre outras possíveis métricas.

# Objetivos do Projeto:

1. **Desenvolver um modelo capaz de prever a estrutura secundária de proteínas** a partir de suas sequências de aminoácidos.
2. Utilizar técnicas de **Machine Learning** e **Deep Learning**, em particular **redes LSTM**, que são adequadas para processar dados sequenciais.
3. **Automatizar a análise de proteínas**: Auxiliar em pesquisas biomédicas ao oferecer uma maneira eficiente de prever a estrutura de proteínas, o que pode acelerar a compreensão de funções proteicas e o desenvolvimento de medicamentos.

# Aplicações Potenciais:
- **Descoberta de novos medicamentos**: A previsão da estrutura de proteínas pode ajudar na identificação de alvos proteicos para o desenvolvimento de novas drogas.
- **Biologia computacional**: Ferramentas como esta são essenciais para análises estruturais em larga escala, fornecendo insights sobre como as proteínas funcionam em nível molecular.

# REDE NEURAL

## 1. Importar as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed


## 2. Definir as funções para pré-processamento

In [2]:
# Definir os aminoácidos conhecidos
amino_acids = 'ACDEFGHIKLMNPQRSTVWXY'

# Função para padronizar o comprimento das sequências
def pad_sequences(sequences, max_length, padding_char='X'):
    return sequences.apply(lambda seq: seq.ljust(max_length, padding_char)[:max_length])

# Função para codificar cada sequência de aminoácidos em uma matriz one-hot
def one_hot_encode_sequence(seq, amino_acids):
    amino_encoder = OneHotEncoder(categories=[list(amino_acids)], sparse_output=False)
    seq_list = list(seq)
    return amino_encoder.fit_transform(np.array(seq_list).reshape(-1, 1))

# Aplicar o one-hot encoding a todas as sequências com padding
def encode_sequences_fixed_length(sequences, amino_acids, max_length):
    sequences_padded = pad_sequences(sequences, max_length)
    seq_encoded = np.array([one_hot_encode_sequence(seq, amino_acids) for seq in sequences_padded])
    return seq_encoded

# Função para padronizar as estruturas secundárias
def pad_sst(ssts, max_length, padding_char='C'):
    return ssts.apply(lambda sst: sst.ljust(max_length, padding_char)[:max_length])


## 3. Carregar os dados e realizar o pré-processamento

In [3]:
# Carregar o conjunto de treinamento
train_data = pd.read_csv('training_secondary_structure_train.csv')  # Substitua pelo seu caminho

# Selecionar as colunas de sequência e estruturas secundárias
seq_train = train_data['seq']
sst3_train = train_data['sst3']

# Definir o comprimento máximo das sequências
MAX_SEQ_LENGTH = 700

# Codificar as sequências de aminoácidos
seq_encoded_train = encode_sequences_fixed_length(seq_train, amino_acids, MAX_SEQ_LENGTH)

# Padronizar as estruturas secundárias
sst3_train_padded = pad_sst(sst3_train, MAX_SEQ_LENGTH)

# Codificar as estruturas secundárias
sst_encoder = LabelEncoder()
sst_encoded_train = np.array([sst_encoder.fit_transform(list(sst)) for sst in sst3_train_padded])

# Dividir os dados em treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(seq_encoded_train, sst_encoded_train, test_size=0.2, random_state=42)

# Exibir as formas dos dados
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)


(6942, 700, 21) (6942, 700) (1736, 700, 21) (1736, 700)


## 4. Definir e compilar o modelo

In [4]:
# Definir o modelo
model = Sequential()
model.add(LSTM(128, input_shape=(700, 21), return_sequences=True))
model.add(TimeDistributed(Dense(3, activation='softmax')))  # 3 classes (H, E, C)

# Compilar o modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


c:\Users\Niciu\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## 5. Treinar o modelo

In [5]:
# Treinar o modelo
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)


Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.7339 - loss: 0.5514 - val_accuracy: 0.7943 - val_loss: 0.3842
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.8063 - loss: 0.3756 - val_accuracy: 0.8229 - val_loss: 0.3666
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - accuracy: 0.8261 - loss: 0.3614 - val_accuracy: 0.8285 - val_loss: 0.3576
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.8307 - loss: 0.3534 - val_accuracy: 0.8292 - val_loss: 0.3562
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - accuracy: 0.8313 - loss: 0.3523 - val_accuracy: 0.8324 - val_loss: 0.3522
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.8327 - loss: 0.3505 - val_accuracy: 0.8323 - val_loss: 0.3519
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - accuracy: 0.8305 - loss: 0.3558 - val_accuracy: 0.8309 - val_loss: 0.3523
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.8301 - loss: 0.3562 - val_accu

## 6. Avaliar o modelo

In [6]:
# Avaliar o modelo
score = model.evaluate(X_val, y_val)
print(f'Acurácia no conjunto de validação: {score[1] * 100:.2f}%')


55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.8353 - loss: 0.3464
Acurácia no conjunto de validação: 83.49%
